In [1]:
import xlrd
import sqlite3
import time

In [2]:
# Open the workbook and define the worksheet
BOOK = xlrd.open_workbook(filename="/Users/muhamuttaqien/Desktop/Al-Qur'an/LQ Tagging/ibnKathirTafsirs DB/IndeksQuranTercanggih.xlsx")

DEPTH_TOPIC_LEVEL = 6

LIST_ENTRY = ["IMAN", "ILMU", "BANGSA-BANGSA TERDAHULU", "SEJARAH", "AL QUR'AN", 
           "AKHLAQ & ADAB", "IBADAH", "MAKANAN & MINUMAN", "PAKAIAN & PERHIASAN", 
           "HUKUM PRIVAT", "MUAMALAT", "PERADILAN & HAKIM", "HUKUM PIDANA, JINAYAH", "JIHAD"]

In [3]:
conn = sqlite3.connect("/Users/muhamuttaqien/Desktop/Al-Qur'an/LQ Tagging/IbnKathirTafsirs-untagged.db3")
c = conn.cursor()

In [4]:
sheet = BOOK.sheet_by_name("IMAN")

### Count rows and columns

In [5]:
table = "tafsir_id_content"
rowsQuery = "SELECT Count() FROM %s" % table
c.execute(rowsQuery)
numberOfRows = c.fetchone()[0]
numberOfRows

6236

In [6]:
columnsQuery = "PRAGMA table_info(%s)" % table
c.execute(columnsQuery)
numberOfColumns = len(c.fetchall())
numberOfColumns

5

In [7]:
nrows = sheet.nrows
nrows

475

In [8]:
ncols = sheet.ncols
ncols

253

### Clear all keywords (optional, hit only once)

In [9]:
for row in range(numberOfRows+1):
    c.execute('UPDATE tafsir_id_content SET keywords = "" WHERE docid = ' + str(row))
    
conn.commit()

### Detect topic and verses

In [10]:
topic = sheet.cell_value(rowx=2, colx=1)
topic

'Keutamaan dan pahala iman'

In [11]:
topic = sheet.cell_value(rowx=361, colx=1)
topic

'Hal-hal yang merusak iman'

In [12]:
verse = sheet.cell_value(rowx=8, colx=7)
verse

'Qs.84:8'

In [13]:
# get verses
for pcol in range(7, ncols):
    colx = pcol
    verse = sheet.cell_value(rowx=2, colx=colx).strip()
    if(verse):
        print(verse)

### Get and Update Parents

In [14]:
def get_and_update_parents(prow, pcol, depth_level, surah, ayah):
    if (depth_level > 1):
        topic_parents = []
        for up_point in range(1, depth_level):
            topic = sheet.cell_value(rowx=(prow-up_point), colx=(pcol-up_point))
            if not topic:
                
                empty_up_point = up_point
                
                while(topic == ''): # check parents if empty looped up
                    topic = sheet.cell_value(rowx=(prow-empty_up_point), colx=(pcol-up_point))
                    empty_up_point += 1
                    
                topic = topic + ' ' + str(depth_level-up_point)
                topic_parents.append(topic)
                
            else:
                topic = topic + ' ' + str(depth_level-up_point)
                topic_parents.append(topic)
        
        topic_parents_string = ', '.join(topic_parents)
                
        print("topic_parents: ", topic_parents_string)
        return topic_parents_string

### Tag topics to verses

In [15]:
def tag_topics(entry, sheet, depth_level):
    
    nrows = sheet.nrows
    ncols = sheet.ncols
    
    for prow in range(nrows):
        pcol = depth_level # index where topics positioned
        # print("prow: ", prow)
        # print("pcol: ", pcol)
        topic = sheet.cell_value(rowx=prow, colx=pcol) # (level 1: Keutamaan dan pahala iman, Cabang-cabang iman, Hal-hal yang merusak iman)
        
        if(topic):
            print("topic: ", topic.strip())
            srow = prow+1
            
            for pcol_v in range(7, ncols):
                colx = pcol_v
                verse = sheet.cell_value(rowx=prow, colx=colx).strip() # (Qs.2:38, Qs.2:62, Qs.2:112)
                if(verse): 
                    # print(verse)
                    # insert verse to db
                    splitted_verse = verse[3:].split(":")
                    surah = splitted_verse[0]
                    ayah = splitted_verse[1]

                    print("Tag into Qs. "+ surah + ":"+ ayah)                      

                    # set multiple tags
                    for row in c.execute('SELECT keywords FROM tafsir_id_content WHERE c0sura = '+ surah +' AND c1aya = ' + ayah):
                        if(row[0] == None or row[0] == ''):
                            topic_parents_string = get_and_update_parents(prow, pcol, depth_level, surah, ayah)
                            topic = (topic + ' ' + str(depth_level)) + (', ' + topic_parents_string)
                            
                            c.execute('UPDATE tafsir_id_content SET keywords = "'+ topic +'" WHERE c0sura = '+ surah +' AND c1aya = ' + ayah)
                        else:
                            topic_parents_string = get_and_update_parents(prow, pcol, depth_level, surah, ayah)
                            topic = (topic + ' ' + str(depth_level)) + (', ' + topic_parents_string)
                            joint_topic = row[0] + ", " + (topic)
                            
                            c.execute('UPDATE tafsir_id_content SET keywords = "'+ joint_topic +'" WHERE c0sura = '+ surah +' AND c1aya = ' + ayah)
                            
    conn.commit()
    print("\n\n---------------------------------- tag %s level %s committed! ----------------------------------\n\n" % (entry, depth_level))

### Commit tagging!

#### all entries

In [16]:
start_time = time.time()

for entry in LIST_ENTRY:
    sheet = BOOK.sheet_by_name(entry)
    
    print("****************************************** %s ******************************************" % (entry))
    
    for level in range(1, (DEPTH_TOPIC_LEVEL+1)):
        tag_topics(entry, sheet, level)
    
print("--- Tagging process takes %.4f seconds ---" % (time.time() - start_time))

****************************************** IMAN ******************************************
topic:  Keutamaan dan pahala iman
topic:  Cabang-cabang iman
topic:  Hal-hal yang merusak iman
topic:  Iman bertambah dan berkurang


---------------------------------- tag IMAN level 1 committed! ----------------------------------


topic:  Kesentosaan orang mukmin di dunia dan di akhirat
Tag into Qs. 2:38
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 2:62
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 2:112
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 2:262
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 2:274
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 2:277
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 3:103
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 3:107
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 3:170
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 4:173
topic_parents:  Keutamaan dan paha

topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 27:50
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 27:53
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 27:57
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 28:7
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 28:9
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 28:20
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 28:25
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 28:31
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 28:35
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 29:15
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 29:24
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 29:32
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 29:33
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 34:20
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 34:21
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 

Tag into Qs. 58:21
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 59:2
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 61:8
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 61:13
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 61:14
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 63:8
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 84:19
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 110:1
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 110:2
topic_parents:  Keutamaan dan pahala iman 1
topic:  Allah memperkokoh orang mukmin
Tag into Qs. 8:11
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 8:12
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 9:26
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 9:40
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 14:27
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 16:102
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 47:7
topi

Tag into Qs. 37:172
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 37:181
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 38:25
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 38:49
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 38:50
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 39:10
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 39:17
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 39:20
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 39:22
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 39:33
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 39:34
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 39:35
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 39:61
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 39:74
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 40:40
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 40:51
topic_parents:  Keutamaan dan paha

topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 26:158
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 26:174
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 26:190
topic_parents:  Keutamaan dan pahala iman 1
topic:  Akidah
topic:  Islam
topic:  Kufur
topic:  Syirik
topic:  Nifak (kemunafikan)
topic:  Bid'ah
Tag into Qs. 16:116
topic_parents:  Hal-hal yang merusak iman 1
Tag into Qs. 57:27
topic_parents:  Hal-hal yang merusak iman 1
topic:  Sihir
topic:  Meramal nasib
Tag into Qs. 7:131
topic_parents:  Hal-hal yang merusak iman 1
Tag into Qs. 27:47
topic_parents:  Hal-hal yang merusak iman 1
Tag into Qs. 36:18
topic_parents:  Hal-hal yang merusak iman 1
Tag into Qs. 36:19
topic_parents:  Hal-hal yang merusak iman 1
topic:  Bertambah dan berkurangnya iman
Tag into Qs. 8:2
topic_parents:  Iman bertambah dan berkurang 1
Tag into Qs. 9:124
topic_parents:  Iman bertambah dan berkurang 1
Tag into Qs. 18:13
topic_parents:  Iman bertambah dan berkurang 1
Tag into Qs.

...

topic_parents:  Nama-nama Allah (Asma'ul Husna) 5, Tentang nama-nama Allah 4, Beriman pada Allah Ta'ala 3, Akidah 2, Cabang-cabang iman 1
topic:  Al Maula (Maha Penolong)
Tag into Qs. 2:286
topic_parents:  Nama-nama Allah (Asma'ul Husna) 5, Tentang nama-nama Allah 4, Beriman pada Allah Ta'ala 3, Akidah 2, Cabang-cabang iman 1
Tag into Qs. 3:150
topic_parents:  Nama-nama Allah (Asma'ul Husna) 5, Tentang nama-nama Allah 4, Beriman pada Allah Ta'ala 3, Akidah 2, Cabang-cabang iman 1
Tag into Qs. 6:62
topic_parents:  Nama-nama Allah (Asma'ul Husna) 5, Tentang nama-nama Allah 4, Beriman pada Allah Ta'ala 3, Akidah 2, Cabang-cabang iman 1
Tag into Qs. 8:40
topic_parents:  Nama-nama Allah (Asma'ul Husna) 5, Tentang nama-nama Allah 4, Beriman pada Allah Ta'ala 3, Akidah 2, Cabang-cabang iman 1
Tag into Qs. 9:51
topic_parents:  Nama-nama Allah (Asma'ul Husna) 5, Tentang nama-nama Allah 4, Beriman pada Allah Ta'ala 3, Akidah 2, Cabang-cabang iman 1
Tag into Qs. 10:30
topic_parents:  Nama-nama Al

topic_parents:  Nama-nama Allah (Asma'ul Husna) 5, Tentang nama-nama Allah 4, Beriman pada Allah Ta'ala 3, Akidah 2, Cabang-cabang iman 1
Tag into Qs. 11:12
topic_parents:  Nama-nama Allah (Asma'ul Husna) 5, Tentang nama-nama Allah 4, Beriman pada Allah Ta'ala 3, Akidah 2, Cabang-cabang iman 1
Tag into Qs. 12:66
topic_parents:  Nama-nama Allah (Asma'ul Husna) 5, Tentang nama-nama Allah 4, Beriman pada Allah Ta'ala 3, Akidah 2, Cabang-cabang iman 1
Tag into Qs. 17:65
topic_parents:  Nama-nama Allah (Asma'ul Husna) 5, Tentang nama-nama Allah 4, Beriman pada Allah Ta'ala 3, Akidah 2, Cabang-cabang iman 1
Tag into Qs. 28:28
topic_parents:  Nama-nama Allah (Asma'ul Husna) 5, Tentang nama-nama Allah 4, Beriman pada Allah Ta'ala 3, Akidah 2, Cabang-cabang iman 1
Tag into Qs. 33:3
topic_parents:  Nama-nama Allah (Asma'ul Husna) 5, Tentang nama-nama Allah 4, Beriman pada Allah Ta'ala 3, Akidah 2, Cabang-cabang iman 1
Tag into Qs. 33:48
topic_parents:  Nama-nama Allah (Asma'ul Husna) 5, Tentang 

topic_parents:  Nama-nama Allah (Asma'ul Husna) 5, Tentang nama-nama Allah 4, Beriman pada Allah Ta'ala 3, Akidah 2, Cabang-cabang iman 1
topic:  Pertemuan Jibril dan Nabi pada saat awal turun wahyu
Tag into Qs. 26:193
topic_parents:  Pertemuan Jibril dan Nabi 5, Jibril as. 4, Beriman kepada Malaikat 3, Akidah 2, Cabang-cabang iman 1
Tag into Qs. 73:1
topic_parents:  Pertemuan Jibril dan Nabi 5, Jibril as. 4, Beriman kepada Malaikat 3, Akidah 2, Cabang-cabang iman 1
Tag into Qs. 74:1
topic_parents:  Pertemuan Jibril dan Nabi 5, Jibril as. 4, Beriman kepada Malaikat 3, Akidah 2, Cabang-cabang iman 1
Tag into Qs. 81:23
topic_parents:  Pertemuan Jibril dan Nabi 5, Jibril as. 4, Beriman kepada Malaikat 3, Akidah 2, Cabang-cabang iman 1
Tag into Qs. 96:1
topic_parents:  Pertemuan Jibril dan Nabi 5, Jibril as. 4, Beriman kepada Malaikat 3, Akidah 2, Cabang-cabang iman 1
Tag into Qs. 96:2
topic_parents:  Pertemuan Jibril dan Nabi 5, Jibril as. 4, Beriman kepada Malaikat 3, Akidah 2, Cabang-ca

Tag into Qs. 56:39
topic_parents:  Perpecahan umat Islam di akhir zaman 5, Tanda-tanda kiamat kecil 4, Beriman pada Hari Akhir 3, Akidah 2, Cabang-cabang iman 1
Tag into Qs. 56:40
topic_parents:  Perpecahan umat Islam di akhir zaman 5, Tanda-tanda kiamat kecil 4, Beriman pada Hari Akhir 3, Akidah 2, Cabang-cabang iman 1
topic:  Perlunya saling menasehati antara sesama
Tag into Qs. 2:206
topic_parents:  Perpecahan umat Islam di akhir zaman 5, Tanda-tanda kiamat kecil 4, Beriman pada Hari Akhir 3, Akidah 2, Cabang-cabang iman 1
Tag into Qs. 49:9
topic_parents:  Perpecahan umat Islam di akhir zaman 5, Tanda-tanda kiamat kecil 4, Beriman pada Hari Akhir 3, Akidah 2, Cabang-cabang iman 1
Tag into Qs. 49:10
topic_parents:  Perpecahan umat Islam di akhir zaman 5, Tanda-tanda kiamat kecil 4, Beriman pada Hari Akhir 3, Akidah 2, Cabang-cabang iman 1


---------------------------------- tag IMAN level 6 committed! ----------------------------------


****************************************** IL

TypeError: must be str, not NoneType

#### One by one

In [ ]:
LIST_ENTRY = ["IMAN", "ILMU", "BANGSA-BANGSA TERDAHULU", "SEJARAH", "AL QUR'AN", 
           "AKHLAQ & ADAB", "IBADAH", "MAKANAN & MINUMAN", "PAKAIAN & PERHIASAN", 
           "HUKUM PRIVAT", "MUAMALAT", "PERADILAN & HAKIM", "HUKUM PIDANA, JINAYAH", "JIHAD"]

start_time = time.time()
entry = "JIHAD"
sheet = BOOK.sheet_by_name(entry)

for level in range(1, (DEPTH_TOPIC_LEVEL+1)):
    tag_topics(entry, sheet, level)

print("--- Tagging process takes %.4f seconds ---" % (time.time() - start_time))